# Street Pavement Rating

This Python script visualizes NYC street pavement ratings for ZIP code 11201 using Altair and GeoPandas.

## Data

This visualization involves two datasets:
* [Street pavement rating](https://data.cityofnewyork.us/Transportation/Street-Pavement-Rating/6yyb-pb25) dataset.
* [Zip code boundaries](https://data.cityofnewyork.us/Business/Zip-Code-Boundaries/i8iw-xf4u)

Due to the large amount of data involved, we will focus on visualizing the street pavement rating for a single zip code region, 11201. 

* `zip_11201` contains the zip boundary data for the target region.
* `road_11201` contains the street pavement rating data for the target region.

## Visual Encoding:
Stroke color represents the pavement rating, using a teal color scale.
Stroke width is thicker for lower ratings, making poor-quality roads more visually prominent.

## Base Map: 
Displays ZIP code 11201 with a light gray background for contrast.

## Interactive Tooltips: 
Show street names and ratings on hover.

## Final Output: 
A 1000x600 resolution map combining street ratings with a ZIP code boundary, improving urban infrastructure insights.

In [1]:
import altair as alt
import pandas as pd
import geopandas as gpd

# Retrieving the street pavement rating dataset
url = "https://data.cityofnewyork.us/api/geospatial/6yyb-pb25?accessType=DOWNLOAD&method=export&format=GeoJSON"
data = gpd.read_file(url)
data = data.astype({"manualrati": int})

# Retrieving the zip boundary dataset
url = "https://raw.githubusercontent.com/qnzhou/practical_data_visualization_in_python/master/module_8_interactive_data_visualization/data/zip_code_040114.geojson"
zip = gpd.read_file(url)
zip = zip.to_crs("EPSG:4326")

zip_11201 = zip[zip.ZIPCODE=="11201"]

In [2]:
road_11201 = data[data.geometry.intersects(zip_11201.unary_union)]

/var/folders/zp/c2nkgdbd34d7z5bjvn8dsy000000gn/T/ipykernel_12509/483156540.py:1: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  road_11201 = data[data.geometry.intersects(zip_11201.unary_union)]


In [3]:
# Define the color scale to be visually salient for lower ratings
color_scale = alt.Scale(domain=[1, 10], scheme='teals')

# Calculate stroke width based on the rating: lower ratings get a thicker stroke
stroke_width_scale = alt.Scale(domain=[1, 10], range=[16, 1])

In [4]:
# Create the base map for the ZIP code
base_map = alt.Chart(zip_11201).mark_geoshape(
    fill='#f0f0f0',
    stroke='grey'
).encode(
    tooltip=['ZIPCODE:N', 'PO_NAME:N']
).properties(
    width=1000,
    height=600,
    title="NYU Brooklyn Campus - ZIP Code 11201"
)

In [5]:
# Create the road rating overlay with the proper legend
roads = alt.Chart(road_11201).mark_geoshape(
    strokeCap='round'
).encode(
    stroke=alt.Color('manualrati:Q', scale=color_scale, legend=alt.Legend(title='Manual Rating')),
    strokeWidth=alt.Size('manualrati:Q', scale=stroke_width_scale),
    tooltip=['onstreetna:N', 'manualrati:Q']
).properties(
    width=1000,
    height=600
)

In [6]:
# Combine the base map and roads layer
chart = base_map + roads

chart

alt.LayerChart(...)